<a href="https://colab.research.google.com/github/frychicken/VantineRemover/blob/main/app_remove_watermarked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/DmitryUlyanov/deep-image-prior

In [ ]:
!mv deep-image-prior/* ./

In [ ]:
from google.colab import files
import os

os.makedirs('sample_data', exist_ok=True)

print("Please upload exactly one file.")

uploaded = files.upload()

if len(uploaded) == 1:
  filename = next(iter(uploaded))
  os.rename(filename, 'data/inpainting/mask.png')
  print(f"The file has been renamed and moved to: sample_data/mask.png")
else:
  print("Error: Please upload exactly one file. Try again.")


In [ ]:
from google.colab import files
import os

os.makedirs('sample_data', exist_ok=True)

print("Please upload exactly one file.")

uploaded = files.upload()

if len(uploaded) == 1:
  filename = next(iter(uploaded))
  os.rename(filename, 'data/inpainting/convertimg.png')
  print(f"The file has been renamed and moved to: sample_data/convertimg.png")
else:
  print("Error: Please upload exactly one file. Try again.")


In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
import torch
import torch.optim

from utils.inpainting_utils import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

PLOT = True
imsize = -1
dim_div_by = 64

In [ ]:

img_path  = 'data/inpainting/convertimg.png'
mask_path = 'data/inpainting/mask.png'


NET_TYPE = 'skip_depth6'

In [ ]:
img_pil, img_np = get_image(img_path, imsize)
img_mask_pil, img_mask_np = get_image(mask_path, imsize)

In [ ]:
img_pil, img_np = get_image(img_path, imsize)
img_mask_pil, img_mask_np = get_image(mask_path, imsize)

In [ ]:
img_mask_var = np_to_torch(img_mask_np).type(dtype)

plot_image_grid([img_np, img_mask_np, img_mask_np*img_np], 3,11);

In [ ]:
pad = 'reflection' # 'zero'
OPT_OVER = 'net'
OPTIMIZER = 'adam'

In [ ]:
import torch
INPUT = 'noise'
input_depth = 32
LR = 0.01
num_iter = 5001
param_noise = False
show_every = 50
figsize = 5
reg_noise_std = 0.03

net = skip(input_depth, img_np.shape[0],
            num_channels_down = [128] * 5,
            num_channels_up   = [128] * 5,
            num_channels_skip = [128] * 5,
            upsample_mode='nearest', filter_skip_size=1, filter_size_up=3, filter_size_down=3,
            need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU').type(dtype)

net = net.type(dtype)
net_input = get_noise(input_depth, INPUT, img_np.shape[1:]).type(dtype)

In [ ]:
# Compute number of parameters
s  = sum(np.prod(list(p.size())) for p in net.parameters())
print ('Number of params: %d' % s)

# Loss
mse = torch.nn.MSELoss().type(dtype)

img_var = np_to_torch(img_np).type(dtype)
mask_var = np_to_torch(img_mask_np).type(dtype)


In [ ]:
i = 0
def closure():

    global i

    if param_noise:
        for n in [x for x in net.parameters() if len(x.size()) == 4]:
            n = n + n.detach().clone().normal_() * n.std() / 50

    net_input = net_input_saved
    if reg_noise_std > 0:
        net_input = net_input_saved + (noise.normal_() * reg_noise_std)


    out = net(net_input)
    # Assuming 'out' is your output tensor
    out = out[:, :, :550, :475]  # Crop the tensor to 550x475

    total_loss = mse(out * mask_var, img_var * mask_var)
    total_loss.backward()

    print ('Iteration %05d    Loss %f' % (i, total_loss.item()), '\r', end='')
    if  PLOT and i % show_every == 0:
        out_np = torch_to_np(out)
        plot_image_grid([np.clip(out_np, 0, 1)], factor=figsize, nrow=1)

    i += 1

    return total_loss

net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()

p = get_params(OPT_OVER, net, net_input)
optimize(OPTIMIZER, p, closure, LR, num_iter)

In [ ]:
out_np = torch_to_np(net(net_input))
plot_image_grid([out_np], factor=5);